# Summary

This demo shows how to:
* Deploy a model with Seldon and access it through an ingress gateway
* Add Dex authentication to that gateway and access the model with authentication
* Use a Canary Rollout with Seldon and Istio to split predictions across multiple models

This demo is modified from [this tutorial](https://docs.seldon.io/projects/seldon-core/en/latest/examples/istio_canary.html)

# Setup

Bootsrap a Juju controller on a Kubernetes cluster, such as shown [here](https://juju.is/docs/olm/microk8s) using Microk8s

Deploy the Seldon and Istio charms, defining a default-gateway for Istio and providing the name of that gateway to Seldon:

In [67]:
gateway_name = "seldon-gateway"
model_name = "seldon-demo"

In [ ]:
!juju add-model $model_name

!juju deploy istio-gateway istio-ingressgateway --trust --kind=ingress
!juju deploy istio-pilot --trust --config default-gateway=$model_name/$gateway_name
!juju relate istio-pilot:istio-pilot istio-ingressgateway:istio-pilot

!juju deploy seldon-core --config istio-gateway=kubeflow/kubeflow-gateway

Wait for everything to deploy and settle, then get the gateway IP

In [ ]:
# sudo snap install juju-wait
!juju wait -vw

In [69]:
gateway_ip=!kubectl get svc istio-ingressgateway -o yaml -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
gateway_ip = gateway_ip[0]

## Helpers

In [8]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, "w") as f:
        f.write(cell.format(**globals()))

# Deploy a Seldon Model and Access it through an Ingress Gateway

### Define and deploy the model

Let's deploy a mock classifier provided by seldon, which will take ndarrays of data and return results.

In [35]:
seldon_deployment_name = "sd-example"

In [29]:
%%writetemplate model.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  labels:
    app: seldon
  name: {seldon_deployment_name}
spec:
  name: example
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/mock_classifier:1.7.0
          imagePullPolicy: IfNotPresent
          name: classifier
        terminationGracePeriodSeconds: 1
    graph:
      children: []
      endpoint:
        type: REST
      name: classifier
      type: MODEL
    name: main
    replicas: 1

In [33]:
!kubectl create -f model.yaml

seldondeployment.machinelearning.seldon.io/sd-example created


In [64]:
jsonpath="'{.items[0].metadata.name}'"
deployment_name = !kubectl get deploy -l seldon-deployment-id=$seldon_deployment_name -o jsonpath=$jsonpath
deployment_name = deployment_name[0]

In [65]:
!kubectl rollout status deploy/$deployment_name

deployment "sd-example-main-0-classifier" successfully rolled out


### Connect to the deployed model

#### Using the Seldon Client

In [70]:
from seldon_core.seldon_client import SeldonClient

sc = SeldonClient(
    gateway="istio",
    deployment_name=seldon_deployment_name,
    namespace=model_name,
    gateway_endpoint=gateway_ip
)


DEBUG:seldon_core.seldon_client:Configuration:{'gateway': 'istio', 'transport': 'rest', 'namespace': 'seldon-demo', 'deployment_name': 'sd-example', 'payload_type': 'tensor', 'gateway_endpoint': '10.64.140.43', 'microservice_endpoint': 'localhost:5000', 'grpc_max_send_message_length': 4194304, 'grpc_max_receive_message_length': 4194304, 'channel_credentials': None, 'call_credentials': None, 'debug': False, 'client_return_type': 'dict', 'ssl': None}


In [72]:
r = sc.predict(gateway="istio", transport="rest")
if r.success, r.response
    print("Congratulations, prediction returned response:")
    print(r.response)
else:
    raise ValueError("Something went wrong - is the gateway set up correctly?")

DEBUG:seldon_core.seldon_client:URL is http://10.64.140.43/seldon/seldon-demo/sd-example/api/v1.0/predictions
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 10.64.140.43:80
DEBUG:urllib3.connectionpool:http://10.64.140.43:80 "POST /seldon/seldon-demo/sd-example/api/v1.0/predictions HTTP/1.1" 200 156
DEBUG:seldon_core.seldon_client:Raw response: {"data":{"names":["proba"],"tensor":{"shape":[1,1],"values":[0.12186205110659878]}},"meta":{"requestPath":{"classifier":"seldonio/mock_classifier:1.7.0"}}}



Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.9417526445253931
  }
}

Response:
{'data': {'names': ['proba'], 'tensor': {'shape': [1, 1], 'values': [0.12186205110659878]}}, 'meta': {'requestPath': {'classifier': 'seldonio/mock_classifier:1.7.0'}}}


In [76]:
assert r.success, "Something went wrong - is the gateway set up correctly?"

#### Using curl

Through `curl`, you can access the classifier deployed by seldon a number of ways:
* direct to the servce:

In [81]:
jsonpath = "'{.items[0].spec.clusterIP}'"
classifier_svc_ip=!kubectl get svc -l seldon-deployment-id=$seldon_deployment_name,seldon.io/model=true -o jsonpath=$jsonpath
classifier_svc_ip = classifier_svc_ip[0]
content_type = "'Content-Type: application/json'"
data = '\'{"data": { "ndarray": [[1]]}}\''

!curl $classifier_svc_ip:9000/predict -X POST -H $content_type -d $data

{"data":{"names":["proba"],"ndarray":[[0.12823373759251927]]},"meta":{"requestPath":{"classifier":"seldonio/mock_classifier:1.7.0"}}}


* via the ingress:

In [82]:
ingress_ip

'10.64.140.43'

In [83]:
!curl $ingress_ip/seldon/$model_name/$seldon_deployment_name/api/v1.0/predictions -X POST -H $content_type -d $data

{"data":{"names":["proba"],"ndarray":[[0.12823373759251927]]},"meta":{"requestPath":{"classifier":"seldonio/mock_classifier:1.7.0"}}}


# Deploy a Canary Model

### Define and deploy the model

In this SeldonDeployment, we define two models, `main` and `canary`, with a traffic split of 75:25, respectively.

In [84]:
seldon_deployment_name = "sd-example-canary"

In [85]:
%%writetemplate canary.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  labels:
    app: seldon
  name: {seldon_deployment_name}
spec:
  name: canary-example
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/mock_classifier:1.7.0
          imagePullPolicy: IfNotPresent
          name: classifier
        terminationGracePeriodSeconds: 1
    graph:
      children: []
      endpoint:
        type: REST
      name: classifier
      type: MODEL
    name: main
    replicas: 1
    traffic: 75
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/mock_classifier:1.7.0
          imagePullPolicy: IfNotPresent
          name: classifier
        terminationGracePeriodSeconds: 1
    graph:
      children: []
      endpoint:
        type: REST
      name: classifier
      type: MODEL
    name: canary
    replicas: 1
    traffic: 25

In [87]:
!kubectl create -f canary.yaml

seldondeployment.machinelearning.seldon.io/sd-example-canary created


In [88]:
jsonpath="'{.items[0].metadata.name}'"
deployment_name = !kubectl get deploy -l seldon-deployment-id=$seldon_deployment_name -o jsonpath=$jsonpath
deployment_name = deployment_name[0]

In [89]:
!kubectl rollout status deploy/$deployment_name

Waiting for deployment "sd-example-canary-canary-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
deployment "sd-example-canary-canary-0-classifier" successfully rolled out


### Connect to the deployed model

#### Using the Seldon Client

Hit the endpoint multiple times so that we can see if it is distributing the load as desired

In [91]:
from seldon_core.seldon_client import SeldonClient

sc = SeldonClient(
    gateway="istio",
    deployment_name=seldon_deployment_name,
    namespace=model_name,
    gateway_endpoint=gateway_ip
)

DEBUG:seldon_core.seldon_client:Configuration:{'gateway': 'istio', 'transport': 'rest', 'namespace': 'seldon-demo', 'deployment_name': 'sd-example-canary', 'payload_type': 'tensor', 'gateway_endpoint': '10.64.140.43', 'microservice_endpoint': 'localhost:5000', 'grpc_max_send_message_length': 4194304, 'grpc_max_receive_message_length': 4194304, 'channel_credentials': None, 'call_credentials': None, 'debug': False, 'client_return_type': 'dict', 'ssl': None}


In [92]:
for _ in range(100):
    r = sc.predict(gateway="istio", transport="rest")
    assert r.success, "Something went wrong - is the gateway set up correctly?"

DEBUG:seldon_core.seldon_client:URL is http://10.64.140.43/seldon/seldon-demo/sd-example-canary/api/v1.0/predictions
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 10.64.140.43:80
DEBUG:urllib3.connectionpool:http://10.64.140.43:80 "POST /seldon/seldon-demo/sd-example-canary/api/v1.0/predictions HTTP/1.1" 200 156
DEBUG:seldon_core.seldon_client:Raw response: {"data":{"names":["proba"],"tensor":{"shape":[1,1],"values":[0.05554955731239537]}},"meta":{"requestPath":{"classifier":"seldonio/mock_classifier:1.7.0"}}}

DEBUG:seldon_core.seldon_client:URL is http://10.64.140.43/seldon/seldon-demo/sd-example-canary/api/v1.0/predictions
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 10.64.140.43:80
DEBUG:urllib3.connectionpool:http://10.64.140.43:80 "POST /seldon/seldon-demo/sd-example-canary/api/v1.0/predictions HTTP/1.1" 200 155
DEBUG:seldon_core.seldon_client:Raw response: {"data":{"names":["proba"],"tensor":{"shape":[1,1],"values":[0.0870091499295114]}},"meta"

We can parse the logs of the `main` and `canary` pods to see how many times each has been hit to demonstrate the traffic split

In [96]:
jsonpath = "'{.items[0].metadata.name}'"
main_count = !kubectl logs $(kubectl get pod -lseldon-app=$seldon_deployment_name-main -o jsonpath=$jsonpath) classifier | grep "root:predict" | wc -l
main_count = main_count[0]

In [97]:
jsonpath = "'{.items[0].metadata.name}'"
canary_count = !kubectl logs $(kubectl get pod -lseldon-app=$seldon_deployment_name-canary -o jsonpath=$jsonpath) classifier | grep "root:predict" | wc -l
canary_count = canary_count[0]

In [98]:
print(f"Number of times main was hit: {main_count}")
print(f"Number of times canary was hit: {canary_count}")

Number of times main was hit: 67
Number of times canary was hit: 33


# Access with Authentication

If we protect our ingress with Dex+OIDC Gatekeeper, unauthenticated access will be rejected.  Below is a method for programatically authenticating and hitting the prediction endpoints.

## Authentication Setup

By adding the `dex-auth` and `oidc-gatekeeper` charms and setting the credentials and `public-url` configurations, we can add authentication to our existing ingress gateway

In [105]:
username="admin"
password="admin"

In [100]:
!juju deploy dex-auth --trust --config static-username=$username --config static-password=$password --config public-url=$ingress_ip
!juju deploy oidc-gatekeeper --config public-url=$ingress_ip

!juju relate dex-auth oidc-gatekeeper
!juju relate dex-auth istio-pilot

Located charm "dex-auth" in charm-hub, revision 78
Deploying "dex-auth" from charm-hub charm "dex-auth", revision 78 in channel 2.28/stable
Located charm "oidc-gatekeeper" in charm-hub, revision 57
Deploying "oidc-gatekeeper" from charm-hub charm "oidc-gatekeeper", revision 57 in channel stable


In [101]:
!juju wait -vw

DEBUG:root:dex-auth/0 workload status is maintenance since 2022-02-25 22:20:46+00:00
DEBUG:root:dex-auth/0 juju agent status is executing since 2022-02-25 22:20:46+00:00
DEBUG:root:oidc-gatekeeper/0 juju agent status is allocating since 2022-02-25 22:20:34+00:00
DEBUG:root:oidc-gatekeeper/0 workload status is waiting since 2022-02-25 22:20:53+00:00
DEBUG:root:dex-auth/0 juju agent status is executing since 2022-02-25 22:20:46+00:00
DEBUG:root:oidc-gatekeeper/0 juju agent status is executing since 2022-02-25 22:20:54+00:00
DEBUG:root:dex-auth/0 workload status is waiting since 2022-02-25 22:20:57+00:00
DEBUG:root:oidc-gatekeeper/0 workload status is waiting since 2022-02-25 22:20:58+00:00
DEBUG:root:dex-auth/0 juju agent status is executing since 2022-02-25 22:20:58+00:00
DEBUG:root:istio-pilot/0 juju agent status is executing since 2022-02-25 22:20:57+00:00
DEBUG:root:oidc-gatekeeper/0 juju agent status is executing since 2022-02-25 22:20:56+00:00
DEBUG:root:istio-pilot/0 workload stat

To authenticate and obtain a authorization cookie, use the `kubeflow_login` helper.  For the url, you can use any valid url that has a VirtualService passing traffic through, such as the models we are serving.

In [103]:
# Helpers for authentication

import logging
import requests
from urllib.parse import parse_qs, urlparse

def kubeflow_login(url, username=None, password=None):
    """Completes the dex/oidc login flow, returning the authservice_session cookie."""
    parsed_url = urlparse(url)
    url_base = f"{parsed_url.scheme}://{parsed_url.netloc}"
    
    data = {
        'login': username or os.getenv('KUBEFLOW_USERNAME', None),
        'password': password or os.getenv('KUBEFLOW_PASSWORD', None),
    }

    if not data['login'] or not data['password']:
        raise ValueError(
            "Missing login credentials - credentials must be passed or defined"
            " in KUBEFLOW_USERNAME/KUBEFLOW_PASSWORD environment variables."
        )

    # GET on url redirects us to the dex_login_url including state for this session
    response = requests.get(
        url,
        verify=False,
        allow_redirects=True
    )
    validate_response_status_code(response, [200], f"Failed to connect to url site '{url}'.")
    dex_login_url = response.url
    logging.debug(f"Redirected to dex_login_url of '{dex_login_url}'")
    
    # Log in, retrieving the redirection to the approval page
    response = requests.post(
        dex_login_url,
        data=data,
        verify=False,
        allow_redirects=False
    )
    validate_response_status_code(
        response, [303], f"Failed to log into dex - are your credentials correct?"
    )
    approval_endpoint = response.headers['location']
    dex_approval_url = url_base + approval_endpoint
    logging.debug(f"Logged in with dex_approval_url of '{dex_approval_url}")
    
    # Get the OIDC approval code and state
    response = requests.get(
        dex_approval_url,
        verify=False,
        allow_redirects=False
    )
    validate_response_status_code(
        response, [303], f"Failed to connect to dex_approval_url '{dex_approval_url}'."
    )
    authservice_endpoint = response.headers['location']
    authservice_url = url_base + authservice_endpoint
    logging.debug(f"Got authservice_url of '{authservice_url}'")

    
    # Access DEX OIDC path to generate session cookie
    response = requests.get(
        authservice_url,
        verify=False,
        allow_redirects=False,
    )
    validate_response_status_code(
        response, [302], f"Failed to connect to authservice_url '{authservice_url}'."
    )
    
    return response.cookies['authservice_session']
    
    
def validate_response_status_code(response, expected_codes: list, error_message: str = ""):
    """Validates the status code of a response, raising a ValueError with message"""
    if error_message:
        error_message += "  "
    if response.status_code not in expected_codes:
        raise ValueError(
            f"{error_message}"
            f"Got response {response.status_code}, expected one of {expected_codes}"
        )


In [106]:
url = f"http://{gateway_ip}/seldon/{model_name}/{seldon_deployment_name}/"
authservice_cookie = kubeflow_login(url, username=username, password=password)

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 10.64.140.43:80
DEBUG:urllib3.connectionpool:http://10.64.140.43:80 "GET /seldon/seldon-demo/sd-example-canary/ HTTP/1.1" 404 19


ValueError: Failed to connect to url site 'http://10.64.140.43/seldon/seldon-demo/sd-example-canary/'.  Got response 404, expected one of [200]

And use the cookie in our seldon client

In [91]:
from seldon_core.seldon_client import SeldonClient

sc = SeldonClient(
    gateway="istio",
    deployment_name=seldon_deployment_name,
    namespace=model_name,
    gateway_endpoint=gateway_ip
)

DEBUG:seldon_core.seldon_client:Configuration:{'gateway': 'istio', 'transport': 'rest', 'namespace': 'seldon-demo', 'deployment_name': 'sd-example-canary', 'payload_type': 'tensor', 'gateway_endpoint': '10.64.140.43', 'microservice_endpoint': 'localhost:5000', 'grpc_max_send_message_length': 4194304, 'grpc_max_receive_message_length': 4194304, 'channel_credentials': None, 'call_credentials': None, 'debug': False, 'client_return_type': 'dict', 'ssl': None}


# Continue with this: 
https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_client.html

In [107]:
r = sc.predict(gateway="istio", transport="rest")
if r.success, r.response
    print("Congratulations, prediction returned response:")
    print(r.response)
else:
    raise ValueError("Something went wrong - is the gateway set up correctly?")

DEBUG:seldon_core.seldon_client:URL is http://10.64.140.43/seldon/seldon-demo/sd-example-canary/api/v1.0/predictions
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 10.64.140.43:80
DEBUG:urllib3.connectionpool:http://10.64.140.43:80 "POST /seldon/seldon-demo/sd-example-canary/api/v1.0/predictions HTTP/1.1" 200 157
DEBUG:seldon_core.seldon_client:Raw response: {"data":{"names":["proba"],"tensor":{"shape":[1,1],"values":[0.055356987829593515]}},"meta":{"requestPath":{"classifier":"seldonio/mock_classifier:1.7.0"}}}

